<a href="https://colab.research.google.com/github/Higgins2718/DS-Unit-2-Sprint-4-Practicing-Understanding/blob/master/FinalKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.6MB/s 


In [0]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor, XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
import math
from sklearn.ensemble import RandomForestClassifier
import datetime


In [0]:
test_features_orig = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQenhVhjm4qtmrTNBjDOmc-OBH3R2C2aSEt2zcc_vi8UrK_PC2UE5z3yoFSL2jlusX1WYBVioGcFCRQ/pub?gid=922169319&single=true&output=csv')
train_features_orig = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTp3w9_zUi31NNzZ0CFrYwlfyS7WwD_mBkGgwWQksXjWhHrqtOWz4DbzkXRqxfkBPMwbcrIY6Ui48zW/pub?gid=326426838&single=true&output=csv')
train_labels_orig = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQOAUDuQIIo4iklp8ZJzJIH2VF_IDL4e1Li_7xg-59ilmh1YhoXU3ymu6tB6EzQUr48dXTzroalfmIQ/pub?gid=26275048&single=true&output=csv')

In [0]:
test_features = test_features_orig
train_features = train_features_orig
train_labels = train_labels_orig

In [0]:
train_features = train_features.fillna('unknown')
test_features = test_features.fillna('unknown')

In [6]:
train_features.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [7]:
train_features.shape

(59400, 40)

In [8]:
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,unknown,GeoData Consultants Ltd,Other,unknown,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,unknown,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [0]:
dates = pd.to_datetime(train_features.date_recorded)
year2000 = pd.to_datetime('2000-01-01')
years = [i.days/365 for i in (dates - year2000)]
train_features.date_recorded = years

dates = pd.to_datetime(test_features.date_recorded)
year2000 = pd.to_datetime('2000-01-01')
years = [i.days/365 for i in (dates - year2000)]
test_features.date_recorded = years

In [10]:
len(train_features)

59400

In [11]:
len(test_features)

14358

In [12]:
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,11.205479,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,13.186301,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,unknown,GeoData Consultants Ltd,Other,unknown,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,13.161644,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,13.084932,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,unknown,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,11.536986,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [0]:
train_features.region_code = train_features.region_code.astype('str')
train_features.district_code = train_features.district_code.astype('str')

test_features.region_code = test_features.region_code.astype('str')
test_features.district_code = test_features.district_code.astype('str')

In [0]:
# Worthless here; basically just one value
train_features = train_features.drop(columns='recorded_by')
test_features = test_features.drop(columns='recorded_by')

In [0]:
# Make sure all vars are formatted properly
type_dict = {'amount_tsh':'float64','date_recorded':'float64',
                 'gps_height':'float64',
                 'longitude':'float64',
                 'latitude':'float64',
                 'num_private':'float64',
                 'population':'float64',
                 'construction_year':'float64',
                 'public_meeting':'str',
                 'permit':'str'}
 
# Mapping
train_features = train_features.astype(dtype = type_dict)
test_features = test_features.astype(dtype = type_dict)

In [0]:
# Necessary for expediting further clearning
numericals = ['amount_tsh',
                    'date_recorded',
                    'gps_height',
                    'longitude',
                    'latitude',
                    'num_private',
                    'population',
                    'construction_year']
# Null island!
null_values = {'amount_tsh':0,
                     'date_recorded':0,
                     'gps_height':0,
                     'longitude':0,
                     'latitude':-2.000000e-08,
                     'num_private':0,
                     'population':0,
                     'construction_year':0}

# replace features meeting above conditions with NaNs
for feature, null in null_values.items():
  train_features[feature] = train_features[feature].replace(null, np.nan)
  
# Now on test data
for feature, null in null_values.items():
  test_features[feature] = test_features[feature].replace(null, np.nan)

In [17]:
len(train_features)

59400

In [18]:
len(test_features)

14358

In [0]:
from sklearn.impute import MissingIndicator


# remove outlier years between 1960 and 2019
 
train_features['construction_year'] = [i if 1960 < i < 2019 else np.nan for i in train_features['construction_year']]
test_features['construction_year'] = [i if 1960 < i < 2019 else np.nan for i in test_features['construction_year']]

# This is super neat!!
indicator = MissingIndicator()
trash_array_train = indicator.fit_transform(train_features[numericals]) # Bool array
trash_array_train = trash_array_train.astype('float64')     # Float64 array

trash_array_test = indicator.fit_transform(test_features[numericals]) # Bool array
trash_array_test = trash_array_test.astype('float64')     # Float64 array

In [0]:
# Use indicator to map values to "trash" features
trashy_names = [numericals[i] + '_trash' for i in indicator.features_]
# Make a trash df, selecting based on names 
trash_df_train = pd.DataFrame(trash_array_train, columns=trashy_names)
trash_df_test = pd.DataFrame(trash_array_test, columns=trashy_names)

# Simple concatenation 
train_features = pd.concat([train_features,trash_df_train], sort=False, axis=1)
train_features = pd.concat([train_features,trash_df_train], sort=False, axis=1)

test_features = pd.concat([test_features,trash_df_test], sort=False, axis=1)
test_features = pd.concat([test_features,trash_df_test], sort=False, axis=1)

In [21]:
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,amount_tsh_trash,gps_height_trash,longitude_trash,latitude_trash,num_private_trash,population_trash,construction_year_trash,amount_tsh_trash,gps_height_trash,longitude_trash,latitude_trash,num_private_trash,population_trash,construction_year_trash
0,69572,6000.0,11.205479,Roman,1390.0,Roman,34.938093,-9.856322,none,NaN,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.0,True,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,8776,NaN,13.186301,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,NaN,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.0,unknown,Other,unknown,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,34310,25.0,13.161644,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,NaN,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.0,True,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,67743,NaN,13.084932,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,NaN,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58.0,True,VWC,unknown,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,19728,NaN,11.536986,Action In A,NaN,Artisan,31.130847,-1.825359,Shuleni,NaN,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,NaN,True,unknown,unknown,True,NaN,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


In [0]:

    
# "ward" info below
# https://en.wikipedia.org/wiki/Ward%27s_method

# Replacing values with means of same ward and map the returned array onto our data
for feature in numericals:
    replacements_train = train_features.groupby('ward')[feature].transform('mean')
    train_features[feature] = train_features[feature].fillna(replacements_train)
    
    replacements_test = test_features.groupby('ward')[feature].transform('mean')
    test_features[feature] = test_features[feature].fillna(replacements_test)
    
# Grouping by region instead of ward
for feature in numericals:
    replacements_train = train_features.groupby('region')[feature].transform('mean')
    train_features[feature] = train_features[feature].fillna(replacements_train)
    
    replacements_test = test_features.groupby('region')[feature].transform('mean')
    test_features[feature] = test_features[feature].fillna(replacements_test)
    
# Taking care of remaining nulls
for feature in numericals:
    replacements_train = train_features[feature].median() 
    train_features[feature] = train_features[feature].fillna(replacements_train)
    
    replacements_test = test_features[feature].median() 
    test_features[feature] = test_features[feature].fillna(replacements_test)

In [23]:
train_features.shape

(59400, 53)

In [24]:
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,amount_tsh_trash,gps_height_trash,longitude_trash,latitude_trash,num_private_trash,population_trash,construction_year_trash,amount_tsh_trash,gps_height_trash,longitude_trash,latitude_trash,num_private_trash,population_trash,construction_year_trash
0,69572,6000.000000,11.205479,Roman,1390.000000,Roman,34.938093,-9.856322,none,24.876777,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.000000,True,VWC,Roman,False,1999.000000,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,8776,542.857143,13.186301,Grumeti,1399.000000,GRUMETI,34.698766,-2.147466,Zahanati,3.857143,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.000000,unknown,Other,unknown,True,2010.000000,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,34310,25.000000,13.161644,Lottery Club,686.000000,World vision,37.460664,-3.821329,Kwa Mahundi,32.395062,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.000000,True,VWC,Nyumba ya mungu pipe scheme,True,2009.000000,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,67743,525.000000,13.084932,Unicef,263.000000,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,47.448276,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58.000000,True,VWC,unknown,True,1986.000000,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,19728,4043.055556,11.536986,Action In A,1236.292754,Artisan,31.130847,-1.825359,Shuleni,28.788889,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,296.596618,True,unknown,unknown,True,1987.601932,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


In [0]:
# Similar to [numericals], a list of our categorical 

# Only train is necessary, since we're simply dealing with columns
categoricals = train_features.select_dtypes(exclude='number').columns.tolist()


In [0]:
train_features[categoricals] = train_features[categoricals].applymap(lambda x: x.lower())
test_features[categoricals] = test_features[categoricals].applymap(lambda x: x.lower())

In [0]:
# NaN values in disguise x-(

nan_list = ['not known','unknown','none','-','##','not kno','unknown installer']
train_features = train_features.replace(nan_list, np.nan)
test_features = test_features.replace(nan_list, np.nan)

In [28]:
len(train_features)

59400

In [29]:
len(test_features)

14358

In [0]:
# Any feature values with fewer than 100 rows gets turned into a NAN
for feature in train_features[categoricals]:
    to_keep = train_features[feature].value_counts()[train_features[feature].value_counts() > 100].index.tolist()
    feature_copy = train_features[feature].copy()
    # if not in to_keep, NaNify
    feature_copy[~feature_copy.isin(to_keep)] = np.nan
    train_features[feature] = feature_copy
    
# Any feature values with fewer than 100 rows gets turned into a NAN
for feature in test_features[categoricals]:
    to_keep = test_features[feature].value_counts()[test_features[feature].value_counts() > 100].index.tolist()
    feature_copy = test_features[feature].copy()
    # if not in to_keep, NaNify
    feature_copy[~feature_copy.isin(to_keep)] = np.nan
    test_features[feature] = feature_copy

In [31]:
train_features[categoricals]

,funder,installer,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,public_meeting,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,roman,NaN,NaN,lake nyasa,NaN,iringa,11,5,ludewa,NaN,true,vwc,roman,false,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,NaN,NaN,zahanati,lake victoria,NaN,mara,20,2,serengeti,NaN,NaN,other,NaN,true,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,NaN,world vision,NaN,pangani,majengo,manyara,21,4,simanjiro,NaN,true,vwc,NaN,true,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,unicef,unicef,NaN,ruvuma / southern coast,NaN,mtwara,90,63,nanyumbu,NaN,true,vwc,NaN,true,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,NaN,artisan,shuleni,lake victoria,NaN,kagera,18,1,karagwe,NaN,true,NaN,NaN,true,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
5,mkinga distric coun,dwe,NaN,pangani,NaN,tanga,4,8,mkinga,NaN,true,vwc,NaN,true,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,NaN,communal standpipe multiple,communal standpipe
6,dwsp,dwsp,NaN,internal,NaN,shinyanga,17,3,shinyanga rural,NaN,true,vwc,NaN,true,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,rwssp,dwe,NaN,lake tanganyika,NaN,shinyanga,17,3,kahama,NaN,true,NaN,NaN,true,nira/tanira,nira/tanira,handpump,wug,user-group,NaN,NaN,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,wateraid,water aid,NaN,lake tanganyika,NaN,tabora,14,6,tabora urban,NaN,true,vwc,NaN,true,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
9,NaN,artisan,NaN,lake victoria,NaN,kagera,18,1,karagwe,NaN,true,NaN,NaN,true,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [0]:
# Fill remaining NaNs
# These features are useless; we can't impute any meaningful values
# Nonetheless, they can't stay NaNs
train_features[categoricals] = train_features[categoricals].fillna('other')
test_features[categoricals] = test_features[categoricals].fillna('other')

In [0]:
# Drop the garbage feature we separated out earlier
garbage = ['longitude','latitude','construction_year_trash',
              'latitude_trash','gps_height_trash',
               'extraction_type_group','extraction_type_class',
               'region_code','waterpoint_type_group','source_type',
              'payment_type','quality_group','quantity_group']
    
train_features = train_features.drop(columns=garbage)
test_features = test_features.drop(columns=garbage)

In [0]:
# We can now supply a meaningful age value, having imputed construction_year NaNs rather well

train_features['age'] = train_features['date_recorded'] - train_features['construction_year']
test_features['age'] = test_features['date_recorded'] - test_features['construction_year']

In [0]:
from sklearn.preprocessing import RobustScaler

# remaining numericals
numericals = ['amount_tsh',
                    'date_recorded',
                    'gps_height',
                    'num_private',
                    'population',
                    'construction_year',
                    'age']
scaler = RobustScaler()
nums_scaled_train = scaler.fit_transform(train_features[numericals])
nums_scaled_train = pd.DataFrame(nums_scaled_train, columns=numericals)
train_features[numericals] = nums_scaled_train


nums_scaled_test = scaler.fit_transform(test_features[numericals])
nums_scaled_test = pd.DataFrame(nums_scaled_test, columns=numericals)
test_features[numericals] = nums_scaled_test

In [0]:
from category_encoders.target_encoder import TargetEncoder
def target_encode_cats(X, X_train, cats, train_labels):
    # Make a clean copy, to ensure we're not changing the original DF
  X2 = X.copy()
    
  y_true = train_labels['status_group']
  y_works = [1.0 if x == 'functional' else 0.0 for x in y_true]
  y_broken = [1.0 if x == 'non functional' else 0.0 for x in y_true]
  y_repair = [1.0 if x == 'functional needs repair' else 0.0 for x in y_true]

  y_vectors = [y_works, y_broken, y_repair]
  X_TE_all = []

  # We want to create encoding based on the training features and 
  # labels, but apply this encoding to any vector (such as X_test)
  for i in [1,2,3]:
      # Make an encoder
      TE = TargetEncoder()
        
      # Fit it to the training data
      TE.fit(X=X_train[cats], y=y_vectors[i-1])

      # Transform the cat columns in X
      X_TE = TE.transform(X2[cats])
        
      # Give them custom names, so that the columns encoded against
      # each target vector have a different name
      X_TE = X_TE.rename(columns=(lambda x: x + '_TE' + str(i)))
      X_TE_all.append(X_TE)

  new_cats = pd.concat(X_TE_all, sort=False, axis=1)
  X2 = X2.drop(columns=cats)
  X2 = pd.concat([X2,new_cats], sort=False, axis=1)
    
  return X2

In [0]:
categoricals = ['funder',
                     'installer',
                     'wpt_name',
                     'basin',
                     'subvillage',
                     'region',
                     'district_code',
                     'lga',
                     'ward',
                     'public_meeting',
                     'scheme_management',
                     'scheme_name',
                     'permit',
                     'extraction_type',
                     'management',
                     'management_group',
                     'payment',
                     'water_quality',
                     'quantity',
                     'source',
                     'source_class',
                     'waterpoint_type',]

In [38]:
train_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [0]:
X_train_temp = train_features
X_train_new = target_encode_cats(X=X_train_temp, 
                                 X_train=X_train_temp, 
                                 cats=categoricals, 
                                 train_labels=train_labels)

In [0]:
#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#X = train_features.select_dtypes(include=numerics)

#X_test = test_features.select_dtypes(include=numerics)
#y = train_labels['status_group']

#from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=.85, test_size=.15)
#X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

In [0]:
test_features = test_features.loc[:,~test_features.columns.duplicated()]

In [0]:
X_train_new = X_train_new.loc[:,~X_train_new.columns.duplicated()]

In [43]:
len(test_features)

14358

In [0]:
#test_features = test_features[test_features.columns]


In [0]:
X_train_new = X_train_new[X_train_new.columns]


In [45]:
len(X_train_new)

59400

In [46]:
%%time
from xgboost import XGBClassifier
modelxgb = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 3, maximize = False, eval_metric = 'merror', eta = .1,
                      max_depth = 14, colsample_bytree = .4)

y_true = train_labels['status_group']
modelxgb.fit(X_train_new, y_true)

CPU times: user 1min 53s, sys: 205 ms, total: 1min 53s
Wall time: 1min 54s


In [47]:
len(test_features)

14358

In [48]:
len(X_train_new)

59400

In [0]:
X_train_new = X_train_new.truncate(after=14357)

In [50]:
len(X_train_new)

14358

In [51]:
len(y_true)

59400

In [0]:
y_true = y_true.truncate(after=14357)

In [53]:
# Test on training data
from sklearn.metrics import accuracy_score
y_pred = modelxgb.predict(X_train_new)
accuracy_score(y_true, y_pred)

0.9323025491015462

In [54]:
test_features.reset_index()

,index,id,amount_tsh,date_recorded,funder,gps_height,installer,wpt_name,num_private,basin,subvillage,region,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,management,management_group,payment,water_quality,quantity,source,source_class,waterpoint_type,amount_tsh_trash,longitude_trash,num_private_trash,population_trash,age
0,0,50785,7.460752,0.176730,other,1.544152,other,other,-4.200000,internal,other,manyara,3,other,other,0.110418,true,parastatal,other,true,1.076923,other,parastatal,parastatal,never pay,soft,seasonal,rainwater harvesting,surface,other,1.0,0.0,1.0,0.0,-1.000776
1,1,51630,1.455898,0.176730,government of tanzania,0.811264,dwe,other,0.000000,pangani,other,arusha,2,arusha rural,other,0.055600,true,vwc,other,true,0.153846,gravity,vwc,user-group,never pay,soft,insufficient,spring,groundwater,communal standpipe,1.0,0.0,1.0,0.0,-0.086616
2,2,17168,0.280866,0.172312,other,0.807831,other,other,-3.240000,internal,other,singida,2,singida rural,other,0.577677,true,vwc,other,other,0.923077,other,vwc,user-group,never pay,soft,insufficient,rainwater harvesting,surface,other,1.0,0.0,1.0,0.0,-0.849042
3,3,45559,-0.462060,0.157585,other,-1.423445,other,other,-4.240000,ruvuma / southern coast,other,lindi,43,other,other,-0.074919,other,vwc,other,true,-0.846154,other,vwc,user-group,other,soft,dry,shallow well,groundwater,other,1.0,0.0,1.0,0.0,0.901012
4,4,49871,0.000000,0.251841,other,0.280907,other,other,-2.879273,ruvuma / southern coast,other,ruvuma,3,mbinga,other,-0.570891,other,water board,other,true,0.153846,gravity,water board,user-group,pay monthly,soft,enough,spring,groundwater,communal standpipe,0.0,0.0,1.0,0.0,-0.075971
5,5,52449,1.455898,0.217968,government of tanzania,1.010362,dwe,other,0.000000,pangani,other,arusha,2,arusha rural,other,-0.205438,true,vwc,other,true,-0.615385,gravity,vwc,user-group,never pay,soft,enough,spring,groundwater,communal standpipe,1.0,0.0,1.0,0.0,0.681029
6,6,24806,-0.497383,-0.861561,government of tanzania,-0.937713,gover,other,22.480000,rufiji,other,iringa,7,other,other,0.838715,true,vwc,other,true,0.692308,india mark ii,vwc,user-group,never pay,salty,enough,machine dbh,groundwater,hand pump,1.0,0.0,1.0,0.0,-0.767018
7,7,28965,-0.480213,0.162003,other,-1.480085,other,other,0.000000,ruvuma / southern coast,other,mtwara,4,other,other,-0.724904,true,water board,borehole,true,-1.230769,submersible,vwc,user-group,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple,1.0,0.0,1.0,0.0,1.282538
8,8,36301,-0.779234,0.159057,unicef,-0.879357,other,other,0.000000,ruvuma / southern coast,other,mtwara,33,other,other,-0.623099,true,vwc,other,false,-0.076923,gravity,vwc,user-group,pay per bucket,soft,insufficient,spring,groundwater,communal standpipe,0.0,0.0,1.0,0.0,0.139420
9,9,54122,-0.798300,0.238586,other,-0.022890,other,other,27.928000,pangani,other,kilimanjaro,7,siha,other,-0.724904,true,water board,other,true,0.384615,gravity,water board,user-group,pay monthly,soft,enough,spring,groundwater,communal standpipe,1.0,0.0,1.0,0.0,-0.306390


In [55]:
len(test_features)

14358

In [0]:
predictions = pd.DataFrame(data={'id': test_features['id'], 'status_group': y_pred})

In [0]:
from google.colab import files
predictions.to_csv('final_model.csv', index=False)
files.download('final_model.csv')